In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,# will return turple(img, label) otherwise dict
    with_info=True# able to get ifro about dataset
)

def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label





In [36]:
for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)


Image shape: (32, 32, 3)
Label: tf.Tensor(7, shape=(), dtype=int64)


In [16]:
AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE = 32


def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))
    
    # if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        # image= tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])
    
    image = tf.image.random_brightness(image, max_delta=0.1)
    
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    
    image = tf.image.random_flip_left_right(image) #50%
    #image = tf.image.random_flip_up_down(image) #50%
    
    return image, label


ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)# другий параметр щоб паралельно запустити
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

        

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)



In [17]:
# print(ds_train.shape)
for image, label in ds_train.take(1):
    print(image.shape)  # Виводить форму зображення
    print(label.shape)  # Виводить форму мітки (label)


(1, 1, 32, 32, 32, 32, 32, 3)
(1, 1, 32, 32, 32)


In [20]:
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(32, 32, 3)),  # specify the input shape
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10)
    ]
)

c:\Users\Lenovo\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)


Epoch 1/5


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, None, None, None, None, 32, 32, 3), dtype=float32). Expected shape (None, 32, 32, 3), but input has incompatible shape (None, None, None, None, None, 32, 32, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, None, None, None, None, 32, 32, 3), dtype=float32)
  • training=True
  • mask=None

In [8]:
# import os

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)


def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE = 32


def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))

    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)

    # a left upside down flipped is still a dog ;)
    image = tf.image.random_flip_left_right(image)  # 50%
    # image = tf.image.random_flip_up_down(image) #%50%

    return image, label


# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
# ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

# TF >= 2.3.0
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Resizing(height=32, width=32,),
        tf.keras.layers.RandomFlip(mode="horizontal"),
        tf.keras.layers.RandomContrast(factor=0.1,),
    ]
)

model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        data_augmentation,
        layers.Conv2D(4, 3, padding="same", activation="relu"),
        layers.Conv2D(8, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)
model.evaluate(ds_test)

Epoch 1/5
1563/1563 - 9s - 6ms/step - accuracy: 0.4012 - loss: 1.6790
Epoch 2/5
1563/1563 - 7s - 5ms/step - accuracy: 0.5149 - loss: 1.3708
Epoch 3/5
1563/1563 - 7s - 4ms/step - accuracy: 0.5557 - loss: 1.2665
Epoch 4/5
1563/1563 - 6s - 4ms/step - accuracy: 0.5795 - loss: 1.2027
Epoch 5/5
1563/1563 - 7s - 4ms/step - accuracy: 0.5969 - loss: 1.1503
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6104 - loss: 1.1232


[1.1216694116592407, 0.6120399832725525]